In [ ]:
import numpy as np
import pandas as pd

from neuralforecast.losses.numpy import mae, rmse


In [ ]:
def _evaluate_ohiot1dm(Y_hat_df, av_mask, model_name):
    results_df = Y_hat_df.copy()
    # Filter values with at least 1 available mask in input window
    results_df = results_df.merge(av_mask[['unique_id', 'cutoff', 'sum_av_mask']], on=['unique_id', 'cutoff'], how='left')
    results_df = results_df[results_df['sum_av_mask'] > 0].reset_index(drop=True)
    # Filter ffill values of y
    results_df = results_df[results_df.available_mask==1]
    # Keep critical values of y
    results_critical_df = results_df[(results_df.y<=70) | (results_df.y>=180)]
    
    return mae(results_df['y'], results_df[model_name]), mae(results_critical_df['y'], results_critical_df[model_name]), rmse(results_df['y'], results_df[model_name]), rmse(results_critical_df['y'], results_critical_df[model_name])

def get_ohiot1dm_results(dataset, trials, model_name, approach):
    mae_group = []
    mae_critical_group = []
    rmse_group = []
    rmse_critical_group = []
    
    for trial in trials:

        if 'exog' in dataset:
            base_df_name = dataset.replace('exog', '')
        else:
            base_df_name = dataset
            
        results = pd.read_csv(f'./results/{dataset}_6/{approach}_models/trial_{approach}_{trial}/forecasts.csv')
        avmask = pd.read_csv(f'./data/{base_df_name}_exog_9_day_test_avmask.csv')
    
        mae_all, mae_critical, rmse_all, rmse_critical = _evaluate_ohiot1dm(results, avmask, model_name)
    
        mae_group.append(mae_all)
        mae_critical_group.append(mae_critical)
        rmse_group.append(rmse_all)
        rmse_critical_group.append(rmse_critical)
    
    mae_group = np.array(mae_group)
    mae_critical_group = np.array(mae_critical_group)
    rmse_group = np.array(rmse_group)
    rmse_critical_group = np.array(rmse_critical_group)
    
    return np.mean(mae_group), np.std(mae_group), np.mean(mae_critical_group), np.std(mae_critical_group), np.mean(rmse_group), np.std(rmse_group), np.mean(rmse_critical_group), np.std(rmse_critical_group)

def get_simglucose_results(dataset, trials, model_name, approach):
    mae_group = []
    mae_critical_group = []
    rmse_group = []
    rmse_critical_group = []
    
    for trial in trials:
    
        results = pd.read_csv(f'./results/{dataset}_6/{approach}_models/trial_{approach}_{trial}/forecasts.csv')
    
        results_critical = results[(results.y<=70) | (results.y>=180)]
    
        mae_all, mae_critical = mae(results.y, results[model_name]), mae(results_critical.y, results_critical[model_name])
        rmse_all, rmse_critical = rmse(results.y, results[model_name]), rmse(results_critical.y, results_critical[model_name])
    
        mae_group.append(mae_all)
        mae_critical_group.append(mae_critical)
        rmse_group.append(rmse_all)
        rmse_critical_group.append(rmse_critical)
    
    mae_group = np.array(mae_group)
    mae_critical_group = np.array(mae_critical_group)
    rmse_group = np.array(rmse_group)
    rmse_critical_group = np.array(rmse_critical_group)

    return np.mean(mae_group), np.std(mae_group), np.mean(mae_critical_group), np.std(mae_critical_group), np.mean(rmse_group), np.std(rmse_group), np.mean(rmse_critical_group), np.std(rmse_critical_group)


## OhioT1DM MAE and RMSE Results

In [ ]:
dataset = 'ohiot1dm'
approach = 'baseline'
model_names = ['AutoNHITS']
trials = range(8)

for model_name in model_names:
    mm, ms, mcm, mcs, rm, rs, rcm, rcs = get_ohiot1dm_results(dataset, trials, model_name, approach)
    final_mae = pd.DataFrame([[mm, ms, mcm, mcs]], index=[model_name]).round(3)
    final_rmse = pd.DataFrame([[rm, rs, rcm, rcs]], index=[model_name]).round(3)
    display(final_mae, final_rmse)

In [ ]:
dataset = 'ohiot1dm_exog'
approach = 'baseline'
model_names = ['AutoNHITS']
trials = range(8)

for model_name in model_names:
    mm, ms, mcm, mcs, rm, rs, rcm, rcs = get_ohiot1dm_results(dataset, trials, model_name, approach)
    final_mae = pd.DataFrame([[mm, ms, mcm, mcs]], index=[model_name]).round(3)
    final_rmse = pd.DataFrame([[rm, rs, rcm, rcs]], index=[model_name]).round(3)
    display(final_mae, final_rmse)

In [ ]:
dataset = 'ohiot1dm_exog'
approach = 'sum_total'
model_names = ['AutoNHITS_TREAT']
trials = range(8)

for model_name in model_names:
    mm, ms, mcm, mcs, rm, rs, rcm, rcs = get_ohiot1dm_results(dataset, trials, model_name, approach)
    final_mae = pd.DataFrame([[mm, ms, mcm, mcs]], index=[model_name]).round(3)
    final_rmse = pd.DataFrame([[rm, rs, rcm, rcs]], index=[model_name]).round(3)
    display(final_mae, final_rmse)

In [ ]:
dataset = 'ohiot1dm_exog'
approach = 'treat'
model_names = ['AutoNHITS_TREAT']
trials = range(8)

for model_name in model_names:
    mm, ms, mcm, mcs, rm, rs, rcm, rcs = get_ohiot1dm_results(dataset, trials, model_name, approach)
    final_mae = pd.DataFrame([[mm, ms, mcm, mcs]], index=[model_name]).round(3)
    final_rmse = pd.DataFrame([[rm, rs, rcm, rcs]], index=[model_name]).round(3)
    display(final_mae, final_rmse)

## Simglucose MAE and RMSE Results

In [ ]:
dataset = 'simglucose'
approach = 'baseline'
model_names = ['AutoNHITS']
trials = range(8)

for model_name in model_names:
    mm, ms, mcm, mcs, rm, rs, rcm, rcs = get_simglucose_results(dataset, trials, model_name, approach)
    final_mae = pd.DataFrame([[mm, ms, mcm, mcs]], index=[model_name]).round(3)
    final_rmse = pd.DataFrame([[rm, rs, rcm, rcs]], index=[model_name]).round(3)
    display(final_mae, final_rmse)

In [ ]:
dataset = 'simglucose_exog'
approach = 'baseline'
model_names = ['AutoNHITS']
trials = range(8)

for model_name in model_names:
    mm, ms, mcm, mcs, rm, rs, rcm, rcs = get_simglucose_results(dataset, trials, model_name, approach)
    final_mae = pd.DataFrame([[mm, ms, mcm, mcs]], index=[model_name]).round(3)
    final_rmse = pd.DataFrame([[rm, rs, rcm, rcs]], index=[model_name]).round(3)
    display(final_mae, final_rmse)

In [ ]:
dataset = 'simglucose_exog'
approach = 'sum_total'
model_names = ['AutoNHITS_TREAT']
trials = range(8)

for model_name in model_names:
    mm, ms, mcm, mcs, rm, rs, rcm, rcs = get_simglucose_results(dataset, trials, model_name, approach)
    final_mae = pd.DataFrame([[mm, ms, mcm, mcs]], index=[model_name]).round(3)
    final_rmse = pd.DataFrame([[rm, rs, rcm, rcs]], index=[model_name]).round(3)
    display(final_mae, final_rmse)

In [ ]:
dataset = 'simglucose_exog'
approach = 'treat'
model_names = ['AutoNHITS_TREAT']
trials = range(8)

for model_name in model_names:
    mm, ms, mcm, mcs, rm, rs, rcm, rcs = get_simglucose_results(dataset, trials, model_name, approach)
    final_mae = pd.DataFrame([[mm, ms, mcm, mcs]], index=[model_name]).round(3)
    final_rmse = pd.DataFrame([[rm, rs, rcm, rcs]], index=[model_name]).round(3)
    display(final_mae, final_rmse)